In [4]:
from sktime.transformers.compose import ColumnConcatenator
from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble
from sktime.classifiers.compose import ColumnEnsembleClassifier
from sktime.classifiers.shapelet_based import ShapeletTransformClassifier
from sktime.datasets import load_basic_motions
from sktime.pipeline import Pipeline

from sktime.classifiers.distance_based import ProximityForest
from sktime.classifiers.distance_based import KNeighborsTimeSeriesClassifier

import numpy as np
import pandas as pd


#cleaning up whatever happened in loading data 
#feed in the data 
from sktime.utils.load_data import from_long_to_nested
import time 

#setting constants 
classifer={
    'TSF_CLF':0,
    'PF_CLF':1,
    'KNN_CLF':2
}


#concatenates ....
#make this assumption cleare 
#write better columns 
#error messages
#define constants when refactoring(readability)

#extracting the data from the csv as a dataframe and reformatting it 

def reformatData(target, file_name):
    raw_df= pd.read_csv(file_name)
    
    #collapses the time cols 
    long_table_df= raw_df.melt(id_vars=["event", "name","start time", "end time","channel"], 
            var_name="anindex", 
            value_name="value")

    sorted_long_table_df=long_table_df.sort_values(by=['event','name','start time','channel'], axis=0)

    unique_dim_ids = sorted_long_table_df.iloc[:, 4].unique()

    for i in range(len(unique_dim_ids)):
        my_channel=unique_dim_ids[i]
        sorted_long_table_df['channel']=sorted_long_table_df['channel'].replace({my_channel:i})
    unique_start_time = sorted_long_table_df.iloc[:, 2].unique()

    for i in range(len(unique_start_time)):
        my_time=unique_start_time[i]
        sorted_long_table_df['start time']=sorted_long_table_df['start time'].replace({my_time:i})

    
    sorted_long_table_df_stripped=sorted_long_table_df.drop(columns=['event','name','end time'])

    sorted_long_table_df_stripped.head()
    df_nested = from_long_to_nested(sorted_long_table_df_stripped)


    target='event'
    new_unique_start_time=sorted_long_table_df.iloc[:, 2].unique()
    labels=[]
    for e in new_unique_start_time:
        x=sorted_long_table_df.loc[sorted_long_table_df['start time']==e,[target]].iloc[0][0]
        labels.append(x)

    np_labels= np.asarray(labels, dtype=np.str)
    
    return df_nested, np_labels 

def splitTestTrain(X, y, percent_train):
    msk = np.random.rand(len(X)) < percent_train
    ytrain=y[msk]
    ytest=y[~msk]
    Xtrain=X[msk]
    Xtest=X[~msk]
    
    return Xtrain, ytrain, Xtest, ytest

def concatenateMethod(Classifier, x_train, y_train, x_test, y_test):
    steps = [
    ('concatenate', ColumnConcatenator()),
    ('classify', Classifier(n_estimators=10))]
    clf = Pipeline(steps)
    clf.fit(x_train, y_train)
    return clf.score(x_test, y_test)

def concatenateMethodTake2(classifier_num, X, y, percent_train):
    concatenation_instance=ColumnConcatenator()
    X_concatenated=concatenation_instance.fit(X).transform(X)
    Xtrain, ytrain, Xtest, ytest= splitTestTrain(X_concatenated,y,percent_train)
    print(Xtrain.head)
    print(Xtrain.shape)

    if(classifier_num==0):
        clf= TimeSeriesForestClassifier(n_estimators=10)
        clf.fit(Xtrain, ytrain)
        return clf.score(Xtest,ytest)
    elif (classifier_num==1):
        knn = KNeighborsTimeSeriesClassifier(metric='dtw')
        knn.fit(Xtrain, ytrain)
        return knn.score(Xtest, ytest)
    elif (classifier_num == 2):    
        pf = ProximityForest(n_trees=10)
        pf.fit(Xtrain, ytrain)
        return pf.score(Xtest, ytest)
    else: 
        return 0
        

In [5]:
target='event'
file_name="../scripts/data/2013e.csv"
X, y=reformatData(target,file_name)
percent_train=0.8
should_profile=True  

FileNotFoundError: [Errno 2] File b'../scripts/data/2013e.csv' does not exist: b'../scripts/data/2013e.csv'

In [3]:
#running time series forest classifier 
acc= concatenateMethodTake2(0, X, y, 0.8)
print(acc)

0.4166666666666667


In [4]:
#k nearest neighbors 
acc= concatenateMethodTake2(1, X[["dim_0", "dim_1","dim_2"]], y, 0.8)

In [5]:
print(acc)

0.4642857142857143


In [ ]:
#proximity forest 
acc= concatenateMethodTake2(2, X[["dim_0", "dim_1"]], y, 0.8)
print(acc)

In [3]:
import time 
def main():
    # this is where method selection will take place and the configuration files will be parsed 
    #numbers of models to run 
    start_time=time.time()
    print("hello")
    end_time= time.time() -start_time
    print(end_time)

In [4]:
main()

hello
0.0003101825714111328


In [2]:
df_example=reformatData("event", "../scripts/data/sample - Sheet1.csv")

In [3]:
df_example[0]

,dim_0,dim_1
0,1 17061.4316 2 17151.1680 3 16959.822...,1 237.3560 2 239.0465 3 235.9601 dtyp...
1,1 16602.2031 2 16672.1426 3 16736.804...,1 230.1286 2 229.4772 3 230.6559 dtyp...
2,1 15789.3135 2 15728.6113 3 15714.095...,1 217.1239 2 216.0460 3 215.5575 dtyp...
3,1 17371.5449 2 17454.6816 3 17400.576...,1 242.5206 2 244.0870 3 242.1639 dtyp...
4,1 16228.7490 2 16249.8633 3 16261.739...,1 223.5448 2 224.1264 3 225.2896 dtyp...


In [4]:
df_example[0].to_csv('example.csv')

In [5]:
df_example[1]

array(['100 0-Back', '100 0-Back', '100 0-Back', '102 2-Back',
       '102 2-Back'], dtype='<U10')

In [2]:
target='event'
file_name="../scripts/data/001-SART-August2017-MB.csv"
X, y=reformatData(target,file_name)
percent_train=0.8
should_profile=True  

In [3]:
acc= concatenateMethodTake2(0, X, y, 0.5)


ValueError: Length mismatch: Expected axis has 700 elements, new values have 175 elements

In [4]:
X.head()

,dim_0,dim_1,dim_2,dim_3
0,1 17704.0 1 17704.0 2 17708.0 2 ...,1 724.17 1 724.17 2 709.00 2 7...,1 563.08 1 563.08 2 543.23 2 5...,1 733.02 1 733.02 2 705.25 2 7...
1,1 17411.0 2 17389.0 3 17370.0 4 ...,1 653.30 2 654.84 3 655.12 4 6...,1 504.03 2 503.39 3 508.28 4 5...,1 647.41 2 648.74 3 666.83 4 6...
2,1 17372.0 2 17327.0 3 17378.0 4 ...,1 662.74 2 669.08 3 674.09 4 6...,1 507.31 2 512.35 3 522.21 4 5...,1 655.47 2 669.55 3 681.86 4 6...
3,1 17714.0 2 17695.0 3 17685.0 4 ...,1 696.93 2 685.00 3 680.29 4 6...,1 538.50 2 525.55 3 519.31 4 5...,1 708.33 2 678.80 3 670.97 4 6...
4,1 17745.0 2 17799.0 3 17832.0 4 ...,1 672.09 2 680.27 3 692.16 4 6...,1 525.06 2 535.19 3 541.90 4 5...,1 680.61 2 699.67 3 712.89 4 7...


In [5]:
X.shape

(175, 4)

In [6]:
acc

NameError: name 'acc' is not defined

In [7]:
steps = [
    ('concatenate', ColumnConcatenator()),
    ('classify', TimeSeriesForestClassifier(n_estimators=100))]
clf = Pipeline(steps)
clf.fit(X, y)
clf.score(X, y)

ValueError: Length mismatch: Expected axis has 700 elements, new values have 175 elements